In [1]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
from numpy.lib.utils import source
import math
import tensorflow as tf
from keras_preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
from keras.models import model_from_json

### Calculate skew angle of an image

In [2]:
def getSkewAngle(cvImage) -> float:
    # Prep image, copy, convert to gray scale, blur, and threshold
    newImage = cvImage.copy()
    gray = cv2.cvtColor(newImage, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9, 9), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
  

    # Apply dilate to merge text into meaningful lines/paragraphs.
    # Use larger kernel on X axis to merge characters into single line, cancelling out any spaces.
    # But use smaller kernel on Y axis to separate between different blocks of text
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=5)

    # Find all contours
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)

    # Find largest contour and surround in min area box
    largestContour = contours[0]
    minAreaRect = cv2.minAreaRect(largestContour)

    # Determine the angle. Convert it to the value that was originally used to obtain skewed image
    angle = minAreaRect[-1]
    if angle < -45:
        angle = 90 + angle
    return -1.0 * angle


### Rotate the image around its center

In [3]:
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

### Deskew image

In [4]:
def deskew(cvImage):
    angle = getSkewAngle(cvImage)

    if(math.fabs(angle)>60):

        angle+=math.degrees(math.pi)/2
    
    return rotateImage(cvImage, -1.0 * angle)

### Calculating Horizontal Projection

In [5]:
def getHorizontalProjectionProfile(image):
  
    # Convert black spots to ones
    image[image == 0]   = 1
    # Convert white spots to zeros
    image[image == 255] = 0
  
    horizontal_projection = np.sum(image, axis = 1) 
  
    return horizontal_projection

### Calculating Vertical Projection

In [6]:
def getVerticalProjectionProfile(image):
  
    # Convert black spots to ones 
    image[image == 0]   = 1
    # Convert white spots to zeros 
    image[image == 255] = 0
  
    vertical_projection = np.sum(image, axis = 0)
  
    return vertical_projection

### Removing Shadows

In [7]:
def shadow_remove(img):
      rgb_planes = cv2.split(img)
      result_norm_planes = []
      for plane in rgb_planes:
          dilated_img = cv2.dilate(plane, np.ones((7,7), np.uint8))
          bg_img = cv2.medianBlur(dilated_img, 21)
          diff_img = 255 - cv2.absdiff(plane, bg_img)
          norm_img = cv2.normalize(diff_img,None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
          result_norm_planes.append(norm_img)
      shadowremov = cv2.merge(result_norm_planes)
      return shadowremov